In [1]:
# Rodrigo Caye Daudt
# rodrigo.cayedaudt@geod.baug.ethz.ch
# 05/2021

import numpy as np
from skimage import io
import os
from sklearn.metrics import confusion_matrix
import sys
from pprint import pprint

if not os.path.exists('./outputs'):
    os.mkdir('./outputs')

print('Imports OK')

Imports OK


In [2]:
# Load images and vectorize

# Ground truth and mask of valid labelled pixels
gt = io.imread('data/test_data.tif')
valid_mask = gt > 0

# Bayes result from L2W2
bayes = io.imread('../L2W2/outputs/02-max_likelihood.png')

# SVM result from L2W2
svm = io.imread('../L2W2/outputs/04-svm.png')

# Extract valid pixels
gt_vec = gt[valid_mask]
bayes_vec = bayes[valid_mask]
svm_vec = svm[valid_mask]

print('Loading images OK')

Loading images OK


In [3]:
bayes_vec.shape
svm_vec.shape
gt_vec.shape

(8210,)

In [4]:
gt_vec, bayes_vec, np.unique(gt_vec, return_counts=True)

(array([4, 4, 4, ..., 7, 7, 7], dtype=uint8),
 array([1, 4, 4, ..., 7, 7, 7], dtype=uint8),
 (array([1, 2, 3, 4, 5, 6, 7, 8], dtype=uint8),
  array([ 962,  876, 1016,  981,  889, 1054, 1169, 1263])))

In [5]:
# Compute confusion matrices using sklearn.metrics.confusion_matrix
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html
# Do not mix y_true and y_pred

cm_bayes = confusion_matrix(gt_vec, bayes_vec, labels=[i for i in range(1, 9)])
cm_svm = confusion_matrix(gt_vec, svm_vec, labels=[i for i in range(1, 9)])
# Confusion matrices are indexed so that you can access values using cm[true_label-1, pred_label-1]

print('Confusion matrices OK')
cm_bayes, cm_svm

Confusion matrices OK


(array([[ 935,    0,    0,    3,    0,   24,    0,    0],
        [   0,  568,    0,    0,    0,  287,   15,    6],
        [   0,    0, 1014,    0,    0,    1,    0,    1],
        [ 110,    0,    0,  844,    7,   20,    0,    0],
        [   0,    0,    0,   10,  831,   48,    0,    0],
        [  64,    0,   27,    3,   18,  883,    1,   58],
        [   0,    0,    0,    0,    0,    0, 1070,   99],
        [   0,    0,  133,    0,    0,   53,   26, 1051]]),
 array([[ 730,   52,    0,    0,    0,  178,    0,    2],
        [   4,  848,    0,    0,    0,   19,    0,    5],
        [   0,    0, 1011,    0,    0,    1,    0,    4],
        [ 320,    0,    0,  352,    0,  309,    0,    0],
        [   0,    0,    0,    0,    0,  874,    0,   15],
        [  34,    0,   34,    0,    0,  591,    1,  394],
        [   0,    0,    0,    0,    0,    0, 1137,   32],
        [   0,    0,   42,    0,    0,  233,   96,  892]]))

`cm[i][j]`: # samples belong to class i, while predicted j

In [6]:
eps = 1e-8
# Calculate producer's, user's, and overall accuracy
# np.diag and np.sum(array, axis=axis) are useful here

# Number of annotated examples per class (TP + FN)
gt_per_class = np.sum(cm_bayes, axis=1) # same for both classes

# True positives per class (TP)
true_positives_bayes = np.diag(cm_bayes)
true_positives_svm = np.diag(cm_svm)

# Number of total predictions per class (TP + FP)
pred_per_class_bayes = np.sum(cm_bayes, axis=0)
pred_per_class_svm = np.sum(cm_svm, axis=0)


# Overall accuracy (one value per classifier)
overall_accuracy_bayes = np.sum(true_positives_bayes) / (np.sum(pred_per_class_bayes) + eps)
overall_accuracy_svm = np.sum(true_positives_svm) / (np.sum(pred_per_class_svm) + eps)

# Producer's accuracy (one value per class per classifier)
producers_accuracy_bayes = np.diag(cm_bayes) / (np.sum((cm_bayes), axis=1) + eps)
producers_accuracy_svm = np.diag(cm_svm) / (np.sum((cm_svm), axis=0) + eps)

# User's accuracy (one value per class per classifier)
users_accuracy_bayes = np.diag(cm_bayes) / (np.sum((cm_bayes), axis=1) + eps)
users_accuracy_svm = np.diag(cm_svm) / (np.sum((cm_svm), axis=1) + eps)


print('Metrics OK')


Metrics OK


In [8]:
pred_per_class_bayes
np.sum((1- producers_accuracy_bayes) * (1 - users_accuracy_bayes))
users_accuracy_bayes, producers_accuracy_bayes

(array([0.97193347, 0.64840183, 0.9980315 , 0.86034659, 0.93475816,
        0.83776091, 0.91531223, 0.83214568]),
 array([0.84310189, 1.        , 0.8637138 , 0.98139535, 0.97079439,
        0.67097264, 0.96223022, 0.86502058]))

In [16]:
(0.876 - 0.012) / (1 - 0.012),np.sum( np.tril(cm_bayes), axis=0)

(0.8744939271255061, array([1109,  568, 1174,  857,  849,  936, 1096, 1051]))

In [35]:
# Print metrics to output file

output_file_name = 'outputs/metrics.txt'
if os.path.exists(output_file_name):
    os.remove(output_file_name)

original_stdout = sys.stdout
with open(output_file_name, 'w') as f:
    sys.stdout = f

    print('Bayes rule classifier\n')
    print(f'Overall accuracy: {overall_accuracy_bayes}')
    print(f'Producer\'s accuracy: {producers_accuracy_bayes}')
    print(f'User\'s accuracy: {users_accuracy_bayes}')
    print(f'Confusion matrix:')
    pprint(cm_bayes)

    print('\n\n' + '='*80)

    print('Support vector machine\n')
    print(f'Overall accuracy: {overall_accuracy_svm}')
    print(f'Producer\'s accuracy: {producers_accuracy_svm}')
    print(f'User\'s accuracy: {users_accuracy_svm}')
    print(f'Confusion matrix:')
    pprint(cm_svm)

    sys.stdout = original_stdout

print(f'Outputs saved to {output_file_name}')

Outputs saved to outputs/metrics.txt


{'forest': 1,
 'water': 2,
 'clouds': 3,
 'fields (green)': 4,
 'fields (brown)': 5,
 'cities': 6,
 'snow': 7,
 'rock': 8}

In [60]:
conf_bayes = cm_bayes - np.diag(np.diag(cm_bayes))
conf_svm = cm_svm - np.diag(np.diag(cm_svm))
conf_bayes, conf_svm
np.max(conf_bayes, axis=1), np.argmax(conf_bayes, axis=1) + 1, conf_bayes

(array([ 24, 287,   1, 110,  48,  64,  99, 133]),
 array([6, 6, 6, 1, 6, 1, 8, 3]),
 array([[  0,   0,   0,   3,   0,  24,   0,   0],
        [  0,   0,   0,   0,   0, 287,  15,   6],
        [  0,   0,   0,   0,   0,   1,   0,   1],
        [110,   0,   0,   0,   7,  20,   0,   0],
        [  0,   0,   0,  10,   0,  48,   0,   0],
        [ 64,   0,  27,   3,  18,   0,   1,  58],
        [  0,   0,   0,   0,   0,   0,   0,  99],
        [  0,   0, 133,   0,   0,  53,  26,   0]]))

In [54]:
np.fill_diagonal(np.zeros((8, 8)), np.diag(cm_bayes))

In [55]:
np.diag(np.diag(cm_bayes))

array([[ 935,    0,    0,    0,    0,    0,    0,    0],
       [   0,  568,    0,    0,    0,    0,    0,    0],
       [   0,    0, 1014,    0,    0,    0,    0,    0],
       [   0,    0,    0,  844,    0,    0,    0,    0],
       [   0,    0,    0,    0,  831,    0,    0,    0],
       [   0,    0,    0,    0,    0,  883,    0,    0],
       [   0,    0,    0,    0,    0,    0, 1070,    0],
       [   0,    0,    0,    0,    0,    0,    0, 1051]])